In [ ]:
import tensorflow as tf
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# 訓練データ
x_train = x_train.reshape(-1, 28, 28, 1) # 4階のテンソルに変換
x_train = x_train.astype('float32')
x_train /= 255
class_num = 10
# 正解ラベルをOne-Hot表現に変換
y_train = tf.keras.utils.to_categorical(y_train, class_num)

# テストデータ
x_test = x_test.reshape(-1, 28, 28, 1) # 4階のテンソルに変換
x_test = x_test.astype('float32')
x_test /= 255
# 正解ラベルをOne-Hot表現に変換
y_test = tf.keras.utils.to_categorical(y_test, class_num)

In [ ]:
class CNN(tf.keras.Model):
    '''
    Attributes:
        conv2D_1(Conv2D):
        conv2D_2(Conv2D):
        flatten(Flatten):
        dorpout(Dropout):
        d1(Dence):
    '''
    def __init__(self):
        super().__init__()
        self.conv2D_1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=(3,3),
            padding='same',
            input_shape=(28, 28,1),
            activation='relu'
        )
        self.conv2D_2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=(3,3),
            padding='same',
            input_shape=(28, 28,1),
            activation='relu'
        )

        self.flatten = tf.keras.layers.Flatten()
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        self.d1 = tf.keras.layers.Dense(10, activation='softmax')

    @tf.function #編集できない関数に機能を追加
    def call(self, x, training=None):
        '''CNNのインスタンスからコールバックされる関数
        Parametes:x(ndarray(float32)):訓練データ
        Returns(float32):CNNの出力として要素数3の1階のテンソル
        '''
        x = self.conv2D_1(x)
        x = self.conv2D_2(x)
        x = self.flatten(x)
        if training:
            x = self.dropout1(x)
        x = self.d1(x)
        return x


In [ ]:
model = Sequential()
# 畳み込み層
model.add(
    Conv2D(filters=32,
    kernel_size=(3, 3),
    padding='same',
    input_shape=(28, 28, 1),
    activation='relu'
    )
)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10,
                activation='softmax'
        ))

In [ ]:
# オブジェクトのコンパイル
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=SGD(lr=0.1),
    metrics=['accuracy']
)
model.summary()

In [ ]:
training_epochs = 100
batch_size = 64

# monitor に設定した値が、 patienceの値の回数続けてmin_delta以上改善しないと、学習をストップ
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1
)

# 学習を行って結果を出力
history = model.fit(
    x_train,
    y_train,
    epochs=training_epochs,
    batch_size=batch_size,
    verbose=1,                  # 学習の進捗状況を出力する
    validation_split=0.2,       # 検証データとして使用する割合
    shuffle=True,               # 検証データを抽出する際にシャッフルする
    callbacks=[early_stopping]  # コールバックはリストで指定する
)
# テストデータで学習を評価するデータを取得
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplots_adjust(wspace=0.2)
plt.subplot(1,2,1)
plt.plot(history.history['loss'],
            label='training',
            color='red')
plt.ylim(0,1)
plt.legend()
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'],
        label='training',
        color='black')
plt.plot(history.history['val_accuracy'],
        label='validation',
        color='red')
plt.ylim(0.5, 1)
plt.legend()
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()